# Advection
Notebook to implement and test different methods on advection type PDEs

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from nodepy import rk
import cvxpy as cp


import numpy.linalg as linalg

from numba import jit, float64, stencil

fe =rk.loadRKM('FE').__num__()
rk4 = rk.loadRKM('RK44').__num__()
rk4x2 = rk4*rk4
ssp2 = rk.loadRKM('SSP22').__num__()
ssp3 = rk.loadRKM('SSP33').__num__()
ssp104 = rk.loadRKM('SSP104').__num__()
merson4 = rk.loadRKM('Merson43').__num__()
bs5 = rk.loadRKM('BS5').__num__()

ck5 = rk.loadRKM('CK5').__num__()
dp5 = rk.loadRKM('DP5').__num__()
pd8 = rk.loadRKM('PD8').__num__()

trbdf = rk.loadRKM('TR-BDF2').__num__()
be = rk.loadRKM('BE').__num__()
irk2 = rk.loadRKM('LobattoIIIA2').__num__()



#Extrapolation method
ex2 = rk.extrap(2,'implicit euler').__num__()
ex3 = rk.extrap(3,'implicit euler').__num__()
ex4 = rk.extrap(4,'implicit euler').__num__()
ex5 = rk.extrap(5,'implicit euler').__num__()
ex6 = rk.extrap(6,'implicit euler').__num__()
ex8 = rk.extrap(8,'implicit euler').__num__()

from OrderCondition import *
from RKimple import *
import utils 

from matplotlib import animation, rc
from IPython.display import HTML

# Linear advection

In [ ]:
N=50
x = np.linspace(0,1,N)
dx = x[1]-x[0]


def f_upwind_advection(t,u):
    dx = 1/len(u)
    du = np.zeros_like(u)
    du[1:] = (u[1:]-u[:-1])
    du[0] = u[0]-u[-1]
    return -du/(dx)


def initial_advection(N,shape = 'delta',scale = 10*np.pi):
    """
    Method to generate test initila data for the advection equation
    
    Parmaters:
    N:     Number of points
    shape: Which testdata 
        'delta':   deltafunction
        'step':    the stepfunction
        'sin':     sine function
        'sin_cut': sine windowed
        'cos_cut': 1-cos windowed
    scale: Scale factor for 'sin' and 'sin_cut' in time
    """
    
    if shape == 'delta':
        u0 = np.zeros(N)
        u0[N//2] = 1
    elif shape == 'step':
        u0 = np.zeros(N)
        u0[N//2:] = 1
    elif shape == 'sin':
        x = np.linspace(0,1,N)
        u0 = np.sin(x*scale)+1
    elif shape == 'sin_cut':
        x = np.linspace(0,1,N)
        u0 = np.zeros(N)
        u0[(x*scale)<=np.pi] = np.sin(x*scale)[(x*scale)<=np.pi]
    elif shape == 'cos_cut':
        x = np.linspace(0,1,N)
        u0 = np.zeros(N)
        u0[(x*scale)<=(2*np.pi)] = 1-np.cos(x*scale)[(x*scale)<=(2*np.pi)]
    else:
        print('wrong shape')             
    
    return u0
    

#u0 = (x<0.5)*1.
#u0 = (x<0.5)+0.2
u0 = (x<0.5)+0.
#u0 = np.sin(2*np.pi*x)+1.

dt = 2.5*dx


def plot_image(u,t):
    cmap=plt.get_cmap('plasma')
    extent=[0,t[-1],1,0]
    plt.imshow(u,cmap=cmap,extent=extent)
    plt.colorbar()
    plt.axis('tight')
    

def plot_at_time(u,T):
    n=np.argmin(np.abs(t-T))
    print(n,t[n])
    plt.plot(u[:,n])
    plt.title('t:'+str(t[n]))
    
    
def init():
    line.set_data([], [])
    time_text.set_text('')
    return (line,)

def animate(i,u,t):
    x = np.linspace(0,1,u.shape[0])
    y = u[:,i]
    line.set_data(x, y)
    time_template = 't = %.3f'
    time_text.set_text(time_template % (t[i]))
    return (line,)

def plot_animate(u,t,frames = 200):
    
    fig, ax = plt.subplots()

    ax.set_xlim(( 0, 1))
    ax.set_ylim((np.min(u), np.max(u)))
    
    global line
    global time_text
    
    time_text = ax.text(0.05, 0.9, '', transform=ax.transAxes)
    line, = ax.plot([], [], lw=2)

    anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=range(0,u.shape[1],max(u.shape[1]//frames,1)), interval=200, 
                               blit=True,fargs = (u,t))
    return HTML(anim.to_jshtml())


    

CFL condition:

Advection coefficent $a$ = 1 

This leads to $$\frac{dx}{dt} \leq 1$$

In [ ]:
N =100

u0 = initial_advection(100,'delta')

methods = (ssp104,dp5,ck5,ssp2,ssp3,merson4,pd8)
dts = utils.findall_dt(RK_variable_b,methods,f_upwind_advection,u0,dt_start=0.009,tol=0.0001,
                   Params=dict(t_final=1,b_fixed=True,fallback = True,num_fallback= 1,
                      LP_opts = {'reduce':True,'feastol':1e-10}))
print(dts)




In [ ]:
utils.plot_times(methods,dts,effective=False,title='Advection linear')

For the linear advection case we can see that the ssp104 ensures positifity for all stable timesteps. 

The dp5 and ck5 methods do not ensure positifity for the hole stable region. The regions with positifity can be extended by adapting the b while enforcing the Order Conditions for 4th Order.

In [ ]:
u0 = initial_advection(100,'delta')

dt = 1/100 *10
dt =0.016
#dt = 0.015 #works for fallback = 1 wyh dp5
#dt = 0.017
#dt = 0.015 #works for fallback = 1 wyh ck5

t,u,b,status = RK_variable_b(pd8,dt,f_upwind_advection,w0=u0,t_final=1,b_fixed=False,return_status=True,
                             fallback = True,num_fallback= 2,LP_opts = {'reduce':True,'feastol':1e-10})
print(status)

In [ ]:
plot_image(u,t)

In [ ]:
plt.plot(t[1:],b.T[1:,:]);

In [ ]:
plot_at_time(u,0.05)

In [ ]:
plot_animate(u,t,frames=len(t))

# Variable Coefficent Advection

$u_t + (a(x,t)u)_x = 0$

$u_t = - (a(x,t)u)_x$

$\frac{\mathrm d}{\mathrm d x} \big( a(x,t)u(x,t) \big) = \frac{\mathrm d}{\mathrm d x} \big( u(x,t) \big) a(x,t) =\frac{\mathrm d}{\mathrm d x} \big( a(x,t) \big) u(x,t)$


Hom. Dirichelt Boundary at $x=0$ open Boundary on $x=1$

In [ ]:


def f_var_coeff_adv(t,u):
    
    #Old implementation, does not enshure conservation
    #A_upwind_advection = 1/dx * (-np.diag(np.ones(N))+np.diag(np.ones(N-1),-1))
    #a = lambda x,t: np.cos(20*x + 45*t)**2
    #a_ = lambda x,t: 2*np.cos(20*x + 45*t)*-np.sin(20*x + 45*t)*20
    #return np.diag(a(x,t))@A_upwind_advection@u +  a_(x,t)*u
    
    N = len(u)
    dx = 1/N
    
    x = np.linspace(0,1,N)
    
    A_upwind_advection = 1/dx * (-np.diag(np.ones(N))+np.diag(np.ones(N-1),-1))
    a = lambda x,t: np.cos(20*x + 45*t)**2
    return A_upwind_advection@(a(x,t)*u)




In [ ]:
N =100

u0 = initial_advection(100,'cos_cut')

methods = (ssp104,dp5,ck5,ssp3,merson4,pd8)
dts = utils.findall_dt(RK_variable_b,methods,f_var_coeff_adv,u0,dt_start=0.009,tol=0.0001,
                   Params=dict(t_final=2,b_fixed=True,fallback = True,num_fallback= 2,
                      LP_opts = {'reduce':True,'feastol':1e-10}))
print(dts)




In [ ]:
utils.plot_times(methods,dts,effective=False,title='variable coefficent advection')

For the variable advection case we can see that the ssp104 does not ensures positifity for all stable timesteps. The region with positife timesteps can be enlarged by adapting the b.

The dp5 and ck5 methods also do not ensure positifity for the hole stable region. When enforcing the Order Conditions for 4th Order $\Delta t_{feasible} = \Delta t_{pos}$. 
When reducing the Conditions to 3d Order $\Delta t_{feasible} > \Delta t_{pos}$. A possible explaination would be that there are not enough degrees of freeedom for the b to find a feasible b. When reducing the Order this increases the degrees of freedom for the the choice of the b. 
This explaination does stil not explain why the linear advection does not show this behavior.

In [ ]:
#dt = 0.09
#dt = 0.10799999999999998
#dt = dt -0.02
#dt = 0.001
#N = 500
#dt = 0.023625 +0.002
dt = 0.028 
N =100
u0 = initial_advection(N,'cos_cut')
t,u,b,KK,status = RK_variable_b(dp5,dt,f_var_coeff_adv,w0=u0,t_final=2,b_fixed=False,fallback = True,num_fallback=1,
                      dumpK=True, LP_opts = {'reduce':True,'feastol':1e-10},return_status=True)
display(status)

In [ ]:
plot_image(u,t)

In [ ]:
plt.plot(t[1:],b.T[1:,:]);

In [ ]:
plot_at_time(u,1.8)

In [ ]:
plot_animate(u,t,frames=300)

In [ ]:
plt.pcolor(KK[1],cmap=plt.get_cmap('bwr'),vmin=-0.5, vmax=0.5)
plt.colorbar()